# PDF Splitter

### Overview:
The puropose of this notebook is to take a conjoined PDF that containes multiple report cards, where each report card has a specified amount of pages and split them into individual pages. Once split, the report cards are renamed based on Last_First_UID.pdf, and placed in the correct directory. The Cells below are used to accomplish this functionality.

### Import Statements

In [ ]:
!pip3 install PyPDF2
!pip install pdfplumber
!pip install pdf2image
!pip install python-poppler
!pip install pytesseract
!pip install PyMuPDF
!sudo apt install tesseract-ocr
!pip install pdf2jpg
# !pip install ironpdf
# from ironpdf import *
from pdf2jpg import pdf2jpg
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image
import fitz
from pdf2image import convert_from_path
import pdfplumber
from PyPDF2 import PdfWriter, PdfReader
import re
import os
import pandas as pd
import requests
import httplib2
import gdown
from apiclient import discovery
from google.oauth2 import service_account
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import gspread
from google.auth import default

  Using cached python_poppler-0.4.1.tar.gz (138 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Configuration Variables

##### Purpose:
Folder_ID:
The folder ID is the end of the URL of the parent folder that you want to use in google drive

Spreadsheet_ID:
The ID of the master spreadsheet that has name and UID

In [ ]:
school = "Cristo Rey Brooklyn High School"
year = 2024
pdf_len = 1
imentor_profile = '/content/drive/MyDrive/_test iMentor Profiles'
imentor_pdf = '/content/drive/MyDrive/_RC 2024 - MP1'

# SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file']
# folder_id = "13PqiFH-iFmza5WSbmdE8F8DWD3zAwnbg"
# SAMPLE_SPREADSHEET_ID_input = "1T_rP5qUWfGXTPVHyCoSz2wi__KMc5JCvYNv910l46P4"

## Important Funtions
I have included 3 different funtions that play an important role in the execution of this notebook.

#### Get_Files_In_Folder:
This does exactly what it sounds like. It a recursive funtion that continually goes deeper into a parent folder tree to extract all the files that may be within that folder structure. As you would have guessed we are using this function to get all the PDF's within a parent folder.

#### Compare_List:
This funtion is being used once we have pulled the data from a single page in a PDF and we want to find the first and last name of a student. It starts by getting the data from Google Sheets, specifically First/Last name and Uinique Identifier. We then percede to find the first and last name in the pdf for each name in the list.

#### Get_Google_Sheets:
This is the function that was referenced above. It is being used to connect to Google Sheets and pull out the first few columns of data.

In [ ]:
def get_files_in_folder(folder_id, service):
    query = f"'{folder_id}' in parents and trashed=false"
    response = service.files().list(q=query, fields='files(name,id,mimeType,webViewLink,createdTime,modifiedTime)').execute()
    files = response.get('files', [])
    dfs = [pd.DataFrame(files)]
    for file in files:
        if file['mimeType'] == 'application/vnd.google-apps.folder':
            dfs.append(get_files_in_folder(folder_id=file['id'], service=service))
    return pd.concat(dfs, ignore_index=True)


def get_files_in_folder_drive(directory):
    pdf_dict = {}
    for root, dirs, files in os.walk(directory,topdown=True):
        #
        # pdf_list = [file for file in files if '.pdf' in file]
        for file in files:
            if 'pdf' in file[-3:]:
                # pdf_list.append(root +'/' +file)
                pdf_dict[root+'/'+file] = root
    return pdf_dict



In [ ]:
print(get_files_in_folder_drive(imentor_pdf))

{'/content/drive/MyDrive/_RC 2024 - MP1/12th Q1 Report Cards (2023-2024).pdf': '/content/drive/MyDrive/_RC 2024 - MP1'}


#### Compare Lists

In [ ]:
def Compare_List(text):
    first = []
    last = []
    uid = []
    first_dict = {}
    data = Get_Google()
    first_list = list(data['First Name'])
    last_list = list(data["Last Name"])
    uid_list = list(data['User ID'])
    for student in first_list:
        if student in text:
            first_dict[student] = first_list.index(student)
    for index in first_dict.keys():
        if last_list[first_dict[index]] in text:
            first.append(index)
            last.append(last_list[first_dict[index]])
            uid.append(uid_list[first_dict[index]])
    if len(first) > 1:
        print("there are multiple matches")
    elif len(first) < 1:
        print(text)
        raise Exception("No Matches Found")
    name_str = last[0]+","+first[0]+" "+"("+str(uid[0])+")"
    pdf = last[0]+","+first[0]+" "+"("+str(uid[0])+")"
    return name_str,pdf

##### Compare_List Test

In [ ]:
text = "Zoey French"
Compare_List(text)

Zoey French


Exception: No Matches Found

#### Get Google Sheet

In [ ]:
def Get_Google_Sheet():
    Secret_File = os.path.join(os.getcwd(), 'ssp-key.json')
    global values_input, service
    creds = service_account.Credentials.from_service_account_file(filename=Secret_File, scopes=SCOPES)
    service = discovery.build('sheets', 'v4', credentials=creds)
    range_name = 'A:D'
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID_input,  range=range_name).execute()
    values_input = result_input.get('values', [])
    data = pd.DataFrame(values_input[1:], columns=values_input[0])
    return(data)

def Get_Google():
    file_loc = "/content/drive/MyDrive/_RC 2024 - MP1/SSP 2024-2027 student info.xlsx"
    df = pd.read_excel(file_loc, index_col=None, usecols="A,G,H")
    return df

##### Get_Google Sheet Test

In [ ]:
gg = Get_Google()
print(gg['First Name'][0] + str(gg['User ID'][0]))


Abdoul269837


## The Program Flow

In [ ]:
def test_library(root):
    text = []
    for root, dirs, files in os.walk('{}/Photos'.format(root),topdown=True):
    # doc = fitz.open(file) # open a document
    # for page in doc: # iterate the document pages
    #   text = page.get_text() # get plain text encoded as UTF-8
    #   print(text)

        for file in files:
            if '.pdf' in file:
            # pdf_list.append(root +'/' +file)
                text.append(pytesseract.image_to_string(root +'/' +file))
            print(text)
    return text


In [ ]:

def pdf_image(file, root):

    outputpath = "{}/Photos".format(root)
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)
    # pdf = PdfDocument.FromFile(file)


    # Extract all pages to a folder as image files
    # pdf.RasterizeToImageFiles("{}/Photos/*.png".format(root),DPI=96)

    result = pdf2jpg.convert_pdf2jpg(file,outputpath, pages="ALL")
    return test_library(root)
    # Store Pdf with convert_from_path function
    # images = convert_from_path(file)
    # print("Checkpoint-----")

    # for i in range(len(images)):
    #     images[i].save('./page'+ str(i) +'.jpg', 'JPEG')
    # test_library('./page1.jpg')

In [ ]:
df = get_files_in_folder_drive(imentor_pdf)
df

{'/content/drive/MyDrive/_RC 2024 - MP1/12th Q1 Report Cards (2023-2024).pdf': '/content/drive/MyDrive/_RC 2024 - MP1'}

In [ ]:
# get_files()
df = get_files_in_folder_drive(imentor_pdf)
j = 0
temporarytext = []
for file in df.keys():#os.listdir("./temp_dir"):
    j += 1
    # if 'pdf' not in file:
    #     continue
    texts = pdf_image(file, df[file])
    print(texts)
    # inputpdf =pdfplumber.open(file)
    # print(inputpdf.pages[0].extract_text())
    inputpdf = PdfReader(open(file, "rb"))
    for i in range(0,len(inputpdf.pages),pdf_len):
        text = texts[i]
        temporarytext.append(text)
        output = PdfWriter()
        output2 = PdfWriter()
        # print(text)
        output.add_page(inputpdf.pages[i])
        output2.add_page(inputpdf.pages[i])
        # text = inputpdf.pages[i].extract_text()
        # student_dict = Get_Str_Info(text)
        try:
            name,pdf = Compare_List(text)
        except Exception as e:
            path = "{}/{}/{}/{}/{}".format(imentor_profile, 'Error', year,school, j)
            if not os.path.exists(path):
                os.makedirs(path)
            with open(r"{}/{}/{}/{}/{}/{}.pdf".format(imentor_profile, 'Error', year,school, j, i), "wb") as outputStream:
                output.write(outputStream)
            continue
        path = "{}/{}/{}/{}".format(imentor_profile, year,school, name)
        path2 = "{}/{}/{}/{}".format(imentor_profile, year, school, 'File Dump')
        if not os.path.exists(path):
            os.makedirs(path)
        if not os.path.exists(path2):
            os.makedirs(path2)
        with open(r"{}/{}/{}/{}/{}.pdf".format(imentor_profile, year,school, name, pdf), "wb") as outputStream:
            output.write(outputStream)
        with open(r"{}/{}/{}/{}/{}.pdf".format(imentor_profile, year, school, 'File Dump', pdf), "wb") as outputStream2:
            output2.write(outputStream2)
        # with open(r"{}/{}/{}/{}/{}.pdf".format(imentor_profile, year,school, name, pdf), "wb") as outputStream:
        #     output.write(outputStream)

    shutil.rmtree("{}/Photos".format(df[file]))

['yy"? REY\n\nBrooklyn\n\nHIGH SCHOOL\n\nReport Card: Quarter 1\n\nHamblin, Kareem\nGrade: 12, 2023-2024\n\nCristo Rey Brooklyn High School\n710 E. 37th Street\nBrooklyn, New York 11203\n(718) 455-3555\nWilliam Henson, President\nDeanna Philippe, Principal\n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\n \n\nCourse Teacher Q1%| Qi T A | Teacher Comments\n\nEnglish IV Klein, Bill 86 B 0 1\n\nSenior Art Fernandez, Romina 91 A- 1 2 Makes an effort to understand the material, a pleasure to have in\nclass.\n\nProbability and Statistics Boukerrou, Kamel 69 F 1 1 Student failed the quizzes and tests. He should take advantage of\noffice hours for assistance and retake the tests. Homework should\nalso be redone for better understanding of the material and better\nGPA.\n\nPhysics Manlangit, Matthew 77 C+ 0 2\n\nEconomics Brady, Michael 76 C 0 1\n\nCWS Work Day - Senior Staff, CWS 98 A 0 0\n\nCollege Counseling Mitchell, Christine 80 B- 0 0\n\nSenior PE & Wellness Mitchell, Dee-Ana 100 A 0 1\n\nTheolo

#### Honarable Mention

This Funtion uses regular expressions to get the student name and information from the PDF

In [ ]:
def Get_Str_Info(text):
    print("----------------------------")
    grade = re.search("^Grade:.*\d", text).group()
    Name = re.search(".*, [a-zA-Z]*", text).group()
    School = re.search(".*High School", text).group()
    year_list = re.findall("20[23][23]", text)
    year = 0
    for i in year_list:
        if int(i) > year:
            year = int(i)
    child_dict = [Name.replace(", ", "_"), grade.replace(": ", "_"), School.replace(" ", "_"), year]
    print(child_dict)
    return child_dict